In [5]:
import rospy
from sensor_msgs.msg import PointCloud2
import numpy as np
import rospy
import tf2_ros
from ros_tensorflow_msgs.srv import *
from rail_segmentation.srv import *
from rail_manipulation_msgs.srv import *
from geometry_msgs.msg import TransformStamped, Pose
from manipulation_test.srv import *
import numpy as np
from ros_numpy import numpify, msgify
import tf
import sys
import moveit_commander
import moveit_msgs.msg
from moveit_msgs.msg import RobotState
from sensor_msgs.msg import JointState
from sensor_msgs import point_cloud2
from sensor_msgs.msg import PointCloud2
import random
from moveit_msgs.msg import AttachedCollisionObject
from moveit_msgs.msg import CollisionObject
from shape_msgs.msg import SolidPrimitive
import uuid
import sensor_msgs.point_cloud2 as pc2

In [6]:
def get_transform_matrix(frame_id_target, frame_id_source):
    tf_buffer = tf2_ros.Buffer()
    tf_listener = tf2_ros.TransformListener(tf_buffer)
    if tf_buffer.can_transform(frame_id_target, frame_id_source, rospy.Time(0), rospy.Duration(4.0)):
        trans = tf_buffer.lookup_transform(frame_id_target, frame_id_source, rospy.Time(0))
    else:
        print("Unable to trans")
    translation = [trans.transform.translation.x, trans.transform.translation.y, trans.transform.translation.z]
    rotation = [trans.transform.rotation.x, trans.transform.rotation.y, trans.transform.rotation.z, trans.transform.rotation.w]
    
    rot_mat = tf.transformations.quaternion_matrix(rotation)

    transform_mat = np.eye(4)
    transform_mat[:3, :3] = rot_mat[:3, :3]
    transform_mat[:3, 3] = translation
    
    return transform_mat

def transform_point_cloud(pc2_msg, target_frame):
    point_cloud_xyz = np.array(list(pc2.read_points(pc2_msg, field_names=("x", "y", "z"), skip_nans=True)))

    point_cloud_xyz_hom = np.c_[point_cloud_xyz, np.ones((len(point_cloud_xyz), 1))]

    transform_mat = get_transform_matrix(target_frame, pc2_msg.header.frame_id)
    
    point_cloud_xyz_transformed_hom = np.dot(transform_mat, point_cloud_xyz_hom.T).T
    
    point_cloud_xyz_transformed = point_cloud_xyz_transformed_hom[:, :3]
    
    header = pc2_msg.header
    header.frame_id = target_frame
    transformed_pc2_msg = pc2.create_cloud_xyz32(header, point_cloud_xyz_transformed)
    
    return transformed_pc2_msg

In [7]:
class ItemPCManager:
    def __init__(self):
        self.color_to_points = {}
        self.color_to_uuid = {}
        self.uuid_to_color = {}
        self.uuid_to_pc2 = {}
        self.color_to_pc2 = {}
    
    def reset(self):
        self.color_to_points = {}
        self.color_to_uuid = {}
        self.uuid_to_color = {}
        self.uuid_to_pc2 = {}
        self.color_to_pc2 = {}
    
    def add_item(self, r, g, b, point, point_cloud2):
        color_key = (r, g, b)
        if color_key not in self.color_to_points:
            self.color_to_points[color_key] = []
            new_uuid = str(uuid.uuid4())
            self.color_to_uuid[color_key] = new_uuid
            self.color_to_pc2[color_key] = point_cloud2
            self.uuid_to_color[new_uuid] = color_key
            self.uuid_to_pc2[new_uuid] = point_cloud2
        if point not in self.color_to_points[color_key]:
            self.color_to_points[color_key].append(point)

    def update_item(self, r, g, b, point):
        color_key = (r, g, b)
        #TODO
        
    def get_item_by_color(self, r, g, b):
        return self.color_to_points.get((r, g, b), [])
    
    def get_pc2_by_color(self, r, g, b):
        color_key = (r, g, b)
        return self.color_to_pc2.get((r, g, b), [])    

    def get_item_by_uuid(self, input_uuid):
        color_key = self.uuid_to_color.get(input_uuid)
        if color_key:
            return self.color_to_points[color_key]
        else:
            return []
    
    def get_pc2_by_uuid(self):
        print("")
        # TODO
    
    def list_items(self):
        for color, points in self.color_to_points.items():
            print("Color: {}, UUID: {}, Number of points: {}".format(color, self.color_to_uuid[color], len(points)))
            
manager = ItemPCManager()

In [9]:
should_continue = True

def filter_point_cloud_by_color(point_cloud_data, color, point_step):
 
    return False

def get_color_from_point(point, is_bigendian):
    color_data = int(point[-1][-1])

    if is_bigendian:
        # Big endian
        r = (color_data >> 24) & 0xFF
        g = (color_data >> 16) & 0xFF
        b = (color_data >> 8) & 0xFF
    else:
        # Little endian
        r = color_data & 0xFF
        g = (color_data >> 8) & 0xFF
        b = (color_data >> 16) & 0xFF

    return (r, g, b)

def parse_points(data, point_step):
    points = []
    for i in range(0, len(data), point_step):
        point_data = data[i:i+point_step]
        x, y, z = struct.unpack_from('fff', point_data, 0)

        rgba = struct.unpack_from('I', point_data, 16)[0]
        r = (rgba >> 16) & 0xFF
        g = (rgba >> 8) & 0xFF
        b = rgba & 0xFF
        a = (rgba >> 24) & 0xFF 
        points.append((x, y, z, r, g, b, a))  
    return points

def callback(point_cloud_data):
    global should_continue
    pc_arr = numpify(point_cloud_data)
    is_bigendian = point_cloud_data.is_bigendian
    point_step = point_cloud_data.point_step
    row_step = point_cloud_data.row_step
    id_dense = point_cloud_data.is_dense
    print("is_bigendian", is_bigendian, "point_step", point_step, "row_step", row_step, "id_dense", id_dense)
    data = point_cloud_data.data
    points = parse_points(data, point_step)
    manager.reset()

    color_to_filtered_points = {}

    color_to_filtered_points[(0, 0, 0)] = []

    for point in points:
        x, y, z, r, g, b, a = point
        color_key = (r, g, b)

        if color_key not in color_to_filtered_points:
            color_to_filtered_points[color_key] = []
        color_to_filtered_points[color_key].append((x, y, z, r, g, b, a))


    for color, color_points in color_to_filtered_points.items():

        if color != (0, 0, 0):
            filtered_pc2 = filter_point_cloud_by_color(point_cloud_data, color, point_step)
        else:
            filtered_pc2 = point_cloud_data

        for p in color_points:
            x, y, z, r, g, b, a = p
            manager.add_item(r, g, b, (x, y, z), filtered_pc2)

    should_continue = False
    subscriber.unregister()

def listener():
    global subscriber
    global should_continue
    print("Listening...")
    rospy.init_node('point_cloud_listener', anonymous=True)
    subscriber = rospy.Subscriber('/head_camera/depth_seg/points', PointCloud2, callback)
    rate = rospy.Rate(1)
    while should_continue:
        break;
        rate.sleep()

In [10]:
listener()

Listening...
('is_bigendian', False, 'point_step', 32, 'row_step', 20480, 'id_dense', False)


In [ ]:
manager.list_items()

Color: (0, 0, 0), UUID: dbd4bdf8-695f-4160-b9c6-4ffa5f8bb1c7, Number of points: 95513


In [ ]:
point_info = manager.get_pc2_by_color(97, 111, 67)

In [8]:
# tf_listener = tf.TransformListener()
# tf_listener.waitForTransform('/base_link','/head_camera_rgb_optical_frame',rospy.Time(), rospy.Duration(4.0))
# trans = tf_listener.lookupTransform('/base_link', '/head_camera_rgb_optical_frame', rospy.Time(0))

tfBuffer = tf2_ros.Buffer()
listener_tf = tf2_ros.TransformListener(tfBuffer)

rate = rospy.Rate(10.0) 
while not rospy.is_shutdown():
    try:
        now = rospy.Time.now()
        past = now - rospy.Duration(5.0) 
        if tfBuffer.can_transform('base_link', 'head_camera_rgb_optical_frame', now):
            camera_trans = tfBuffer.lookup_transform('base_link', 'head_camera_rgb_optical_frame', now)

            break
        else:
            rospy.loginfo("Waiting for transform")
    except (tf2_ros.LookupException, tf2_ros.ConnectivityException, tf2_ros.ExtrapolationException) as ex:
        rospy.loginfo("Transform error: %s", ex)
    rate.sleep()

[INFO] [1700609806.171839, 179.556351]: Waiting for transform
[INFO] [1700609806.211241, 179.656357]: Waiting for transform


In [9]:
camera_trans

header: 
  seq: 0
  stamp: 
    secs: 179
    nsecs: 756362915
  frame_id: "base_link"
child_frame_id: "head_camera_rgb_optical_frame"
transform: 
  translation: 
    x: 0.165375168117
    y: 0.02
    z: 1.32031320838
  rotation: 
    x: -0.640856361209
    y: 0.640856361208
    z: -0.298836283436
    w: 0.298836283439

In [46]:
transed_p = transform_point_cloud(point_info, 'base_link')

In [47]:
rospy.wait_for_service('grasp_predict')
grasp_predictor = rospy.ServiceProxy('grasp_predict', Predict)

In [49]:
try:
    predicted_grasp_result = grasp_predictor(point_info, point_info, camera_trans)
except rospy.ServiceException as e:
    print("Service call failed: %s"%e)

In [50]:
predicted_grasp_result

predicted_grasp_poses: 
  - 
    header: 
      seq: 0
      stamp: 
        secs: 0
        nsecs:         0
      frame_id: "base_link"
    pose: 
      position: 
        x: 0.239806564461
        y: -0.0286752025804
        z: 2.08361434953
      orientation: 
        x: -0.0535973045353
        y: 0.964351623221
        z: 0.258567260032
        w: 0.0172118499243
  - 
    header: 
      seq: 0
      stamp: 
        secs: 0
        nsecs:         0
      frame_id: "base_link"
    pose: 
      position: 
        x: 0.238257944829
        y: 0.0629892130178
        z: 1.9782480071
      orientation: 
        x: -0.065364316324
        y: 0.974300215012
        z: 0.215456271825
        w: 0.00672250777646
  - 
    header: 
      seq: 0
      stamp: 
        secs: 0
        nsecs:         0
      frame_id: "base_link"
    pose: 
      position: 
        x: 0.207695093292
        y: 0.13673119217
        z: 1.8963066664
      orientation: 
        x: -0.0960714896832
        y: 0.9446

In [51]:
moveit_commander.roscpp_initialize(sys.argv)

In [52]:
from visualization_msgs.msg import Marker, MarkerArray
from std_msgs.msg import Header, ColorRGBA
from geometry_msgs.msg import Vector3

marker_array_publisher = rospy.Publisher('/grasp_markers', MarkerArray, queue_size=10)

def publish_markers(predict_response):
    marker_array_msg = MarkerArray()
    marker_array_publisher.publish(marker_array_msg)
    for i, grasp_pose in enumerate(predict_response.predicted_grasp_poses):
        marker_msg = Marker()
        marker_msg.header.frame_id = grasp_pose.header.frame_id or "base_link"
        marker_msg.header.stamp = rospy.Time.now()
        marker_msg.ns = "grasp_poses"
        marker_msg.id = i
        marker_msg.type = Marker.ARROW
        marker_msg.action = Marker.ADD
        marker_msg.pose = grasp_pose.pose
        
        marker_msg.scale = Vector3(0.01, 0.002, 0.002)

        marker_msg.color = ColorRGBA(0.0, 1.0, 0.0, 1.0)
        
        marker_msg.lifetime = rospy.Duration(0)
        
        marker_array_msg.markers.append(marker_msg)
    
    marker_array_publisher.publish(marker_array_msg)
    rospy.loginfo("Published grasp markers to /grasp_markers")

    

In [53]:
publish_markers(predicted_grasp_result)

[INFO] [1700611773.723127, 3664.949951]: Published grasp markers to /grasp_markers


In [23]:
move_group = moveit_commander.MoveGroupCommander("arm")
scene = moveit_commander.PlanningSceneInterface()
move_group.set_max_velocity_scaling_factor(1.0)

In [24]:
grasp_shift = np.array([[1,0,0,0.02],[0,1,0,0],[0,0,1,0],[0,0,0,1]])
pre_grasp_shift = np.array([[1,0,0,-0.03],[0,1,0,0],[0,0,1,0],[0,0,0,1]])
pick_shift = np.array([[1,0,0,0],[0,1,0,0],[0,0,1,0.05],[0,0,0,1]])

for i in range(len(predicted_grasp_result.predicted_grasp_poses)):
    move_group.set_start_state_to_current_state()
    
    # generate grasp pose
    grasp_pose = numpify(predicted_grasp_result.predicted_grasp_poses[i].pose).dot(grasp_shift)

    # calculate the pre-grasp pose
    pre_grasp_pose = grasp_pose.dot(pre_grasp_shift)
    
    # calculate the pick-up pose
    pick_up_pose = pick_shift.dot(grasp_pose)

    trans = tf.transformations.translation_from_matrix(pre_grasp_pose).tolist()
    quat = tf.transformations.quaternion_from_matrix(pre_grasp_pose).tolist()
    
    #trans = tf.transformations.translation_from_matrix(grasp_pose).tolist()
    #quat = tf.transformations.quaternion_from_matrix(grasp_pose).tolist()

    move_group.clear_pose_targets()
    move_group.set_pose_target(trans + quat)
    plan_result = move_group.plan()
    if plan_result[0]:
        joint_state = JointState()
        joint_state.header.stamp = rospy.Time.now()
        joint_state.name = plan_result[1].joint_trajectory.joint_names
        joint_state.position = plan_result[1].joint_trajectory.points[-1].positions
        moveit_robot_state = move_group.get_current_state()
        moveit_robot_state.joint_state = joint_state
        after_pre_grasp_current_config = moveit_robot_state
        move_group.set_start_state(moveit_robot_state)
        (approach_plan, fraction) = move_group.compute_cartesian_path([msgify(geometry_msgs.msg.Pose, grasp_pose)], 0.01, 0.0)
        # check whether you can approach the object
        if fraction < 0.9:
            continue
        
        moveit_robot_state = RobotState()
        joint_state.header.stamp = rospy.Time.now()
        joint_state.position = approach_plan.joint_trajectory.points[-1].positions
        moveit_robot_state.joint_state = joint_state
        move_group.set_start_state(moveit_robot_state)
        (pick_plan, fraction) = move_group.compute_cartesian_path([msgify(geometry_msgs.msg.Pose, pick_up_pose)], 0.01, 0.0)
        # check whether you can pick the object
        if fraction < 0.9:
            continue
            
        print "got a way to pick up the object"
        
        break

move_group.clear_pose_targets()
scene.clear()